In [1]:
%load_ext autoreload
%autoreload 2

from primitives.primitives import *
import matplotlib.pyplot as plt
from numpy.linalg import eig
import qutip as qt

In [2]:
def commute(a, b):
    a = np.array(a, dtype=np.complex128)
    b = np.array(b, dtype=np.complex128)

    c = a@b
    d = b@a
    if np.allclose(c-d, np.zeros(c.shape)):
        print("Commute")
    elif np.allclose(c+d, np.zeros(c.shape)):
        print("Anti-commute")
    else:
        print("Null")
        
def is_unitary(m):
    return np.allclose(np.eye(m.shape[0]), m.dot(m.conj().T))

In [16]:
S = np.array((TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_5*sy_gamma_2) - TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_5*sy_gamma_1)))

np.all(S.conj().T == S)

True

In [8]:
S = np.array(-1j * (TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_2) - TensorProduct(sy_gamma_1, sy_gamma_5*sy_gamma_2)))

np.all(S.conj().T == S)

True

In [11]:
S = np.array(-1j * (TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_2, sy_gamma_5) - TensorProduct(sy_gamma_1, sy_gamma_5*sy_gamma_2, sy_gamma_5)))

np.all(S.conj().T == S)

True

In [59]:
S = np.array((TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_5*sy_gamma_2) - 
              TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_5*sy_gamma_1)))

np.all(S.conj().T == S)

True

In [4]:
a = TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_2)
b = TensorProduct(sy_gamma_1, sy_gamma_5*sy_gamma_2)

commute(a,b)

Commute


In [18]:
a = (TensorProduct(sy_id, sy_id, sy_id) - TensorProduct(sy_id, sy_gamma_5, sy_id))* (TensorProduct(sy_id, sy_id, sy_id) - TensorProduct(sy_gamma_5, sy_id, sy_id))
b = (TensorProduct(sy_id, sy_id, sy_id) - TensorProduct(sy_id, sy_id, sy_gamma_5))* (TensorProduct(sy_id, sy_id, sy_id) - TensorProduct(sy_id, sy_gamma_5, sy_id))

commute(a, b)

Commute


In [3]:
from scipy.linalg import eig
def find_unitary_transform(A, B):
    """
    Find the unitary matrix U such that U A U^dagger = B.

    Parameters:
    A (ndarray): Input matrix A.
    B (ndarray): Input matrix B.

    Returns:
    U (ndarray or None): Unitary matrix U if it exists, otherwise None.
    """
    A = np.array(A, complex)
    B = np.array(B, complex)
    
    # Get the eigenvalues and eigenvectors of A
    eigvals_A, eigvecs_A = eig(A)
    eigvals_B, eigvecs_B = eig(B)
    
    # Sort the eigenvalues and corresponding eigenvectors for comparison
    idx_A = np.argsort(eigvals_A)
    idx_B = np.argsort(eigvals_B)
    
    eigvals_A_sorted = eigvals_A[idx_A]
    eigvals_B_sorted = eigvals_B[idx_B]
    
    eigvecs_A_sorted = eigvecs_A[:, idx_A]
    eigvecs_B_sorted = eigvecs_B[:, idx_B]

    # Check if the sorted eigenvalues match
    if not np.allclose(eigvals_A_sorted, eigvals_B_sorted):
        print("not all eigenvalues match")
        print(eigvals_A_sorted)
        print(eigvals_B_sorted)
        return None

    eigvecs_A_sorted = np.linalg.qr(eigvecs_A_sorted)[0]
    eigvecs_B_sorted = np.linalg.qr(eigvecs_B_sorted)[0]

    # Construct the unitary matrix U
    U = eigvecs_B_sorted @ eigvecs_A_sorted.conj().T
    
    # Check if U is unitary
    if np.allclose(U @ U.conj().T, np.eye(U.shape[0])):
        return U
    else:
        print("not unitary")
        return U

In [4]:
def to_latex(mat):
    text = r"\begin{pmatrix}"
    text += "\n"
    for row in mat:
        for idx, el in enumerate(row):
            imag = False
            
            if np.imag(el) != 0 and np.real(el) != 0:
                raise Exception
            if np.imag(el) != 0:
                imag = True
                el = np.imag(el)
            else:
                el = np.real(el)
            if el%1 == 0:
                text += str(int(el))
            elif np.abs(el) == 0.7071068:
                if el < 0:
                    text += "-"
                if imag:
                    text += "i"
                text += r"\sqrt{0.5}"
            else:
                text += str(el)
            
            if idx + 1 < len(row):
                text += " & "
        text += r"\\"
        text += "\n"
        

    text += r"\end{pmatrix}"
    return text

### Spinful (2)

\begin{equation}
\begin{split}
    &\exp\left( i\left( \tilde\Gamma^1_{\vec r}\Gamma^2_{\vec{r+x}} - \tilde\Gamma^2_{\vec r}\Gamma^1_{\vec{r+x}}\right)\right)\\
    &=U^\dagger_{(\vec{r,r+x})}\left(\exp\left(\tilde\Gamma_{\vec r} + \tilde\Gamma_{\vec{r+x}}\right)\right) U_{(\vec{r,r+x})}
\end{split}
\end{equation}


In [6]:
from scipy.linalg import expm

In [13]:
# spinful 2 horizontal: U

a = (-25j) * (1j*(TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_2) - TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_1)))
b = (-25j) * (TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5))

a = expm(np.array(a, dtype=np.complex128))
b = expm(np.array(b, dtype=np.complex128))

U = find_unitary_transform(a, b)

if (np.allclose(U @ U.conj().T, np.eye(U.shape[0]), rtol=1e-6)):
    print("Unitary found")
    new_U = np.round(U, 7)
    print(to_latex(new_U))
else:
    print("Unitary not found")

Unitary found
\begin{pmatrix}
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
0 & 

\begin{pmatrix}
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
\end{pmatrix}

In [69]:
# spinful 2 vertical: V

a = TensorProduct(sy_gamma_5*sy_gamma_1, sy_gamma_5*sy_gamma_2) - TensorProduct(sy_gamma_5*sy_gamma_2, sy_gamma_5*sy_gamma_1)
b = TensorProduct(sy_gamma_5, sy_id) + TensorProduct(sy_id, sy_gamma_5)

a = expm(np.array(a, dtype=np.complex128))
b = expm(np.array(b, dtype=np.complex128))

V = find_unitary_transform(a, b)

if (np.allclose(V @ V.conj().T, np.eye(V.shape[0]), rtol=1e-6)):
    print("Unitary found")
    new_V = np.round(V, 7)
    print(to_latex(new_V))
else:
    print("Unitary not found")

Unitary found
\begin{pmatrix}
0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 

In [70]:
# spinful 2 vertical: W

a = TensorProduct(sy_gamma_1, sy_gamma_2)
b = TensorProduct(sy_gamma_5, sy_id)

a = expm(np.array(a, dtype=np.complex128))
b = expm(np.array(b, dtype=np.complex128))

W = find_unitary_transform(a, b)

if (np.allclose(W @ W.conj().T, np.eye(W.shape[0]), rtol=1e-6)):
    print("Unitary found")
    new_W = np.round(W, 7)
    print(to_latex(new_W))
else:
    print("Unitary not found")

Unitary found
\begin{pmatrix}
-i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0\\
0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0\\
i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0\\
0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -\sqrt{0.5} & 0\\
0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0\\
0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0\\
0 & -i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & i\sqrt{0.5} & 0 & 0 & 0 & 0 & 0 & \sqrt{0.5} & 0 & 0\\
0 & 0 & 0 & 0 & 0 & -i\sqrt{0.5} & 0 & 0 & 0 &